## "BRAIN GAINS" BOOKSTORE

In [14]:
from IPython.display import Image, display
image = Image(url='https://static.dezeen.com/uploads/2019/07/chongqing-zhongshuge-bookstore-stepwell-x-living-china_sq-b.jpg')
display(image)

In [7]:
import requests

url = 'https://cae-bootstore.herokuapp.com'

endpoint_login = "/login"
endpoint_user = "/user"
endpoint_book = "/book"

class StoreBooks():
    def __init__(self):
        self.books = requests.get(url + endpoint_book)
      
    def get_books(self):
        return self.books.json()['books']
    
    def get_category_list(self):
        return {book['subject'].title() for book in self.get_books()}
    
    def get_book_by_category(self, category):
        return list(filter(lambda book: book['subject'].title() == category.title(), self.get_books()))
    
    def get_single_book(self, book_id):
        single_book = requests.get(url + endpoint_book + '/' + str(book_id))
        return single_book.json()

store_books = StoreBooks()
books = store_books.get_books()
store_books.get_category_list()

{'Cooking',
 'Entertainment',
 'Marriage',
 'Marriage Advice',
 'Programming',
 'Science',
 'Self Help'}

In [4]:
import json

def register_user(payload):
    payload_json_string = json.dumps(payload)
    headers = {
        'Content-Type':'application/json'
    }
    response = requests.post(
        url + endpoint_user,
        data = payload_json_string,
        headers = headers
    )
    return response.text

import base64

def login_user(user_name, password):
    auth_string = user_name + ":" + password
    
    headers={
        'Authorization' : "Basic "+base64.b64encode(auth_string.encode()).decode()
    }
    
    user_data = requests.get(
        url + endpoint_login,
        headers=headers
    )
    return user_data.json()

def edit_user(token, payload):
    payload_json_string = json.dumps(payload)
    headers={
        'Content-Type':'application/json',
        'Authorization':'Bearer ' + token
    }
    response = requests.put(
        url + endpoint_user,
        data=payload_json_string,
        headers=headers
    )
    return response.text

def delete_user(token):
    headers = {
        'Authorization':"Bearer " + token
    }
    
    response = requests.delete(
        url+endpoint_user,
        headers=headers
    )
    return response.text

In [5]:
from getpass import getpass
import time
from IPython.display import clear_output
from IPython.display import Image
from IPython.display import display


def login(email):
    clear_output()
    password=getpass("Password: ")
    user = login_user(email, password) 
    return user

def register():
    clear_output()
    print("Registration:")
    email = input("Email: ")
    first_name = input("First Name: ")
    last_name = input("Last Name: ")
    password = getpass("Password: ")
    
    user_dict={
        "email":email,
        "first_name":first_name,
        "last_name":last_name,
        "password":password
    }
    return register_user(user_dict)


def display_book_short(book):
        print(f"{book['id']} \t| {book['title'][:50].ljust(50)} | \t{book['subject']}")

def display_book_long(book):
        cover = Image(url=book['img'])
        display(cover)
        print(f'''
Title:\t {book['title']}
Author: {book['author']}
Pages:\t {book['pages']}
Subject: {book['subject']}
Summary: {book['summary']}
''')
    
    
class ReadingList():
    def __init__(self):
        self.reading_list=[]
        
    def add_book(self, book):
        if book not in self.reading_list:
            self.reading_list.append(book)
    
    def remove_book(self, book):
        self.reading_list.remove(book)
    
    def empty(self):
        self.reading_list=[]
    
    def show_book_list(self):
        clear_output()
        if not self.reading_list:
            print("Your reading list is empty")
        for book in self.reading_list:
            print(f'''
{"="*50}            
\n
Title:\t {book['title']}
Book ID: {book['id']}
Author:\t {book['author']}
Subject: {book['subject']}
Summary: {book['summary']}
\n
{"="*50}            
\n
''')



    def show_small_book_list(self):
        if not self.reading_list:
            print("Your reading list is empty")
        else:
            for book in self.reading_list:
                display_book_short(book)
                
reading_list = ReadingList()
# BROWSING BOOKS                

                
def browse_books(books, reading_list, subject=None):
    while True:
        clear_output()
        print(f'''
Welcome to the Book Browser
You are viewing {subject if subject else 'all'} books
[ID] \t| {"[TITLE]".ljust(50)} | [SUBJECT]
        ''')
        if subject:
            books = store_books.get_book_by_category(subject)
        for book in books:
            display_book_short(book)

        selection = input("Select you book by ID [BACK to back out]")
        if selection.lower()=='back':
            return
        elif selection.isnumeric() and int(selection) in map(lambda book: book['id'], books):
            selection=int(selection)
            
            
            while True:
                print(f'''
You Selected: {list(filter(lambda book: book['id'] == selection, books))[0]['title']}
1. Add Book To Reading List
2. View More Information
3. Go Back
4. Go To Main Menu
''')
                def add_book_to_reading_list():
                    clear_output()
                    reading_list.add_book(list(filter(lambda book: book['id'] == selection, books))[0])
                    print("As you wish")
                    time.sleep(1)
                    return
                
                def display_more_information():
                    clear_output()
                    display_book_long(list(filter(lambda book: book['id']==selection, books))[0])
                    input("Press Enter To Continue")
                    return
                    
                
                action = input("Action: ")
                if action =="1":
                    add_book_to_reading_list()
                elif action =="2":
                    display_more_information()
                elif action=="3":
                    break
                elif action=="4":
                    return
                    
        else:
            print("Invalid ID")
            time.sleep(2)
            continue
    

def main():
    store_books = StoreBooks()
    reading_list = ReadingList()


    while True:
        clear_output()
        print("Welcome to the Bookstore")
        email = input("Type your email to login or Type `register` to Register ")
        if email == 'register':
            success_register=register()
            if success_register:
                print("You have successfully registered")
                continue
        elif email.lower() == "quit":
            print("Goodbye")
            break
        else:
            try:
                login(email)
            except:
                print("Invalid Username/Password combo")
                time.sleep(2)
                continue
        # First Scene of our app (above)
        while True:
            clear_output()
            print("""
Welcome to the Repository            
You can:            
1. Browse All Books
2. Browse Book by Category
3. View Reading List
4. Remove Book From Reading List
5. Quit     
""")
            def browse_book_by_category():
                while True:
                    print(" | ".join(store_books.get_category_list()))
                    cat = input("Category: ").title()
                    if cat in store_books.get_category_list():
                        browse_books(books, reading_list, cat)
                        break
                    print("Invalid Category")
                    time.sleep(2)
                    continue
                    
            def remove_booke_from_reading_list():
                while True:
                    clear_output()
                    reading_list.show_small_book_list()
                    garbage = input("What book ID would you like to remove? [BACK to back out]")
                    if garbage.lower() == "back":
                        break
                    elif garbage.isnumeric() and int(garbage) in map(lambda book: book['id'], reading_list.reading_list):
                        reading_list.remove_book(list(filter(lambda book: book['id']==int(garbage), reading_list.reading_list))[0])
                        print(f'{garbage} has been removed')
                        time.sleep(2)
                        break
                    else:
                        print(f'{garbage} is not in your collection')
                        time.sleep(2)
                        break
                  
                
            command = input("Select your Fate: ")
            if command == "1":
                browse_books(books, reading_list)
            elif command == "2":
                browse_book_by_category()
            elif command == "3":
                reading_list.show_book_list()
                input("Press Enter To Return")
                continue
            elif command == "4":
                remove_booke_from_reading_list()
                continue 
            elif command == "5":
                print("Goodbye")
                break
            else:
                print("Invalid Selection")
                time.sleep(2)
                continue
        break
            

main()            

Welcome to the Bookstore


KeyboardInterrupt: Interrupted by user

In [ ]:
cindy@crawford.com

In [11]:
from unittest.mock import patch
from unittest import TestCase, main


class StoreBooks(TestCase):
    def setUp(self):
        self.books = requests.get(url + endpoint_book)

    def test_1_get_category_list(self):
        self.assertTrue(self.get_category_list() == {'Cooking',
 'Entertainment',
 'Marriage',
 'Marriage Advice',
 'Programming',
 'Science',
 'Self Help'})


if __name__ =="__main__":
    main()

E
ERROR: C:\Users\dmitr\AppData\Roaming\jupyter\runtime\kernel-cf0ddb9f-e34b-488e-ab21-478884ef9dd6 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\dmitr\AppData\Roaming\jupyter\runtime\kernel-cf0ddb9f-e34b-488e-ab21-478884ef9dd6'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

C:\Users\dmitr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
pip install testbook

Note: you may need to restart the kernel to use updated packages.


In [1]:
def foo(x):
    return x + 1

In [2]:
from testbook import testbook

@testbook('/path/to/notebook.ipynb', execute=True)
def test_foo(tb):
    foo = tb.ref("foo")

    assert foo(2) == 3

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/notebook.ipynb'